# Assignment 1 - Factory Location Problem
JM0100-M-6 Business Analytics  
Myrthe Wouters  
u1273195

## Factory Location Problem
A company is transitioning into producing a product they formerly bought externally. To that end, it needs to decide where to build its factories. There are $N$ possible locations for a factory. Fixed costs (in EURO) per year for opening a factory at location $i$ are $F(i)$. Each factory can produce at most $C(i)$ kilograms of the product. The total demand per year (in kilograms) for each delivery address (DA) $j$ is $D(j)$. The transport costs (in EURO) for one kilogram between the factories and delivery addresses are $t(i,j)$.

We want to figure out at which locations factories will be opened such that the total costs are minimal?

### Question 1
#### (3 points) Develop a mathematical optimization model to answer this question. Clearly write down what are decision variables, objectives, and constraints.

##### Decision variables  
$y_i = \left\{
    \begin{array}{ll}
        1 & \textrm{if a factory is build at location $i$}\\
        0 & \textrm{otherwise}
    \end{array}
\right.$

$x_{ij}$ is the amount of kilograms supplied by factory $i$ to delivery address $j$.

##### Constraints 

Demand of each delivery address $j$ should be satisfied:    
$\sum_{i=1}^{n}x_{ij} = D_j$ for $j=1, ..., m$
  

Capacity constraint of each factory $i$ should be satisfied:  
$\sum_{j=1}^{m}x_{ij} \leq C_iy_i$ for $i=1, ..., n$

Natural constraints:  
$x_{ij} \geq 0$ for $i=1, ..., n; j=1, ..., m$

$y_i \in \mathbb{B}$ for $i=1, ..., n$

##### Objective function

$f(x,y) = \sum_{i=1}^{n} \sum_{j=1}^{m} t_{ij}x_{ij} + \sum_{i=1}^{n} F_i y_i$

##### Formulation

$
\begin{array}{lll}
     min & \sum_{i=1}^{n} \sum_{j=1}^{m} t_{ij}x_{ij} + \sum_{i=1}^{n} F_i y_i & \\
     s.t. & \sum_{i=1}^{n}x_{ij} = D_j & 1 \leq j \leq m \\
     & \sum_{j=1}^{m}x_{ij} \leq C_iy_i & 1 \leq i \leq n \\ 
     & x_{ij} \geq 0 & 1 \leq j \leq m, 1 \leq i \leq n \\
     & y_i \in \mathbb{B} & 1 \leq i \leq n \\
\end{array}
$


### Question 2
#### (4 points) Solve the model for the following data:
* $N=37$, the possible locations are given in the Excel spreadsheet.
* Set of possible locations for DCs is same as set of DAs.
* Capacity $C(i)$ is 4000 kilograms for all $i$.
* $D(j)$ is $100$ kilograms, for all locations $j$.
* $F(i) = 100,000$ for all locations $i$.
* $t(i,j)$ is equal to the distance between locations ($1$ euro per kilometer per kilogram), for all locations $i$ and The distances between locations are given in the attached Excel spreadsheet.

__What is the objective value? At which locations are factories opened?__

In [1]:
# import necessary packages
import pandas as pd
from pulp import *

In [2]:
# import data
distances = pd.read_excel('Distances.xlsx', index_col='Distances in km')
locations = list(distances.index)
delivery_addresses = list(distances.columns)
C = {i: 4000 for i in locations} # Capacity
D = {j: 100 for j in delivery_addresses} # Demand
F = {i: 100000 for i in locations} # Fixed costs
t = {i: {j: distances.loc[i, j] for j in delivery_addresses} for i in locations} # Transport costs

In [3]:
# General functions for defining and solving the model and for printing the results

## Function for defining the model 

def factory_location_problem(name, dc, da, t, F, D, C, max_nr_factories=None):
    
    # Create the model
    model = LpProblem(name=name, sense=LpMinimize)
    
    # Create list with combinations of all dc and da
    location_delivery = [(i, j) for i in dc for j in da]

    # Set LpVariable parameters
    # Natural constraints included
    y = LpVariable.dicts(name='location', indexs=dc, lowBound=0, upBound=1, cat='Integer')
    x = LpVariable.dicts(name='kilograms', indexs=location_delivery, lowBound=0, cat='Integer')
    
    
    # Add objective function
    model += lpSum(t[i][j] * x[(i,j)] for i in dc for j in da) + lpSum(F[i]*y[i] for i in dc)
    
    # Add constraints
    for j in da:
        model += lpSum(x[(i,j)] for i in dc) == D[j] # demand constraint

    for i in dc:
        model += lpSum(x[(i,j)] for j in da) <= C[i]*y[i] # capacity constraint
        
    # Add extra constraint for the maximum number of factories that can be opened if applies (Question 3)
    if max_nr_factories:
        for i in dc:
            model += lpSum(y[i] for i in dc) <= max_nr_factories 
    
    return model

## Function for solving the model

def solve_model(model):
    model.solve()
    
    # Extract objective value
    total_costs = value(model.objective)
    
    # Extract the locations where factories will be placed, i.e., y_i = 1
    factories = [v.name[len('location_'):] for v in model.variables() 
                 if v.varValue==1 and v.name.startswith('location_')]
    
    # Dictionary with optimal solution
    result = {'Total costs': total_costs, 
              'Factories': factories}
    
    return result

## Function for printing the results

def print_result(solution):
    print('Total costs = {}'.format(solution['Total costs']))
    print('Factories will be opened at the following locations: {}.'.format(', '.join(solution['Factories'])))

In [4]:
# Define model for question 2
model_q2= factory_location_problem('factory-location-problem-q2', locations, delivery_addresses, t, F, D, C)
solution_q2 = solve_model(model_q2) # Solve

print_result(solution_q2)

Total costs = 1450600.0
Factories will be opened at the following locations: Antwerp, Athens, Berlin, Madrid, Rome, Stuttgart, Turin.


### Question 3

#### (a) (1 point) Given the parameters and descriptions in Question 2, we now additionally restrict the number of factories that can be opened to a maximum of 6. Which constraint(s) do you need to add to the original mathematical model?

The following constraint needs to be added to the original mathematical model:

$\sum_{i=1}^{n} y_i \leq 6$ for $i=1,..., n$

So that the new formulation becomes:

$
\begin{array}{lll}
     min & \sum_{i=1}^{n} \sum_{j=1}^{m} t_{ij}x_{ij} + \sum_{i=1}^{n} F_i y_i & \\
     s.t. & \sum_{i=1}^{n}x_{ij} = D_j & 1 \leq j \leq m \\
     & \sum_{j=1}^{m}x_{ij} \leq C_iy_i & 1 \leq i \leq n \\ 
     & \sum_{i=1}^{n} y_i \leq 6 & 1 \leq i \leq n \\
     & x_{ij} \geq 0 & 1 \leq j \leq m, 1 \leq i \leq n \\
     & y_i \in \mathbb{B} & 1 \leq i \leq n \\
\end{array}
$


#### (b) (1 point) Solve this new model. At which locations are factories opened, and what is the objective value?

In [5]:
# Define model for question 3, including the new constraint for the maximum number of factories equal to six
model_q3 = factory_location_problem('factory-location-problem-q3', locations, delivery_addresses, t, F, D, C, 
                                   max_nr_factories=6)
solution_q3 = solve_model(model_q3) # Solve

print_result(solution_q3)

Total costs = 1454000.0
Factories will be opened at the following locations: Antwerp, Athens, Berlin, Genoa, Madrid, Stuttgart.


### Question 4
#### (1 point) At which locations are factories opened when we remove Madrid and Rome as possibilities (they are still used as a delivery address)? Do not enforce the extra restriction from question 3 in answering this question. 

In [6]:
# Remove Madrid and Rome as locations for factory possibility, leave delivery addresses unchanged
locations_q4 = [location for location in locations if location not in ['Rome', 'Madrid']]

# Define model for question 4, excluding the new possible factory locations in Madrid and Rome
model_q4 = factory_location_problem('factory-location-problem-q4', locations_q4, delivery_addresses, t, F, D, C)
solution_q4 = solve_model(model_q4) # Solve

print_result(solution_q4)

Total costs = 1460600.0
Factories will be opened at the following locations: Antwerp, Athens, Berlin, Lisbon, Naples, Stuttgart, Turin.
